# Evaluación de revisión #2 LFYA

In [1]:
!python --version

Python 3.8.15


## 0.1 Install libraries abd set them up

In [2]:
!pip install -e /home/ivanvladimir/projects/maquinas

Obtaining file:///home/ivanvladimir/projects/maquinas
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for maquinas (pyproject.toml) ... done
  Created wheel for maquinas: filename=maquinas-0.1.5.21-0.editable-py3-none-any.whl size=15647 sha256=bbac4fba7c6a870b4a835dcd255550d5dd9f7e1a48000ed38b2ebb940383e8cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-xkmb0f_b/wheels/ae/89/a4/e9d715bf0bebfa5292ad12c08a9f21f24c502506865fb0ac28
Successfully built maquinas
  Attempting uninstall: maquinas
    Found existing installation: maquinas 0.1.5.21
    Uninstalling maquinas-0.1.5.21:
      Successfully uninstalled maquinas-0.1.5.21


In [3]:

!pip install rich
!pip install --upgrade -q gspread
!pip install gspread-formatting
!pip install fuzzywuzzy[speedup]
!pip install gdown
!pip install pandas
!pip install authlib


In [4]:
#from google.colab import drive
from rich.progress import track as progress_bar
from rich.jupyter import print
from rich.console import Console
console = Console(width=200)
import pandas as pd
import gspread
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#from googleapiclient.discovery import build
from gspread_formatting import *
from google.auth import default
from fuzzywuzzy import fuzz, process
import re
import urllib.request
import gdown
from IPython.display import SVG

## 0.2 Reading list and asignation

In [5]:
url_info = "" #@param {"type":"string"}

lista={}
lista_={}
with open("/home/ivanvladimir/Dropbox/Cursos/lfya/2023i/lista.txt") as f:
  for i,line in enumerate(f):
    line=line.strip()
    lista[i]=line.lower()
    lista_[line.lower()]=i

assign="""0,0,5,6,10
1,3,4,6,8
2,0,5,6,10
3,3,4,6,9
4,2,5,6,10
5,2,4,6,10
6,0,5,6,9
7,2,4,6,9
8,3,4,6,9
9,2,5,6,7
10,0,5,6,9
11,3,4,6,8
12,2,4,6,10
13,2,4,6,8
14,0,5,6,10
15,1,4,6,8
16,1,4,6,10
17,2,4,6,9
18,1,4,6,7
19,0,5,6,7
20,1,4,6,10
21,2,4,6,8
22,1,4,6,10
23,2,4,6,7
24,0,4,6,9
25,3,4,6,10
26,1,5,6,8
27,0,5,6,8
28,3,5,6,10
29,0,4,6,9
30,2,5,6,10
31,1,5,6,9
32,3,5,6,8
33,3,4,6,9
34,2,4,6,9
35,0,5,6,7
36,2,5,6,10
37,0,4,6,9
38,0,5,6,8
39,3,5,6,7
40,1,5,6,10
41,0,4,6,10
42,1,5,6,9
""".splitlines()
assign=[l.split(',') for l in assign]
assign=[tuple(int(v) for v in vals[1:]) for vals in assign]

print("\n".join(lista_.keys()))

alan bautista razo
alejandra rosales piña
alexis alejandro sanchez rosas
andres castrejon jimenez
angel isaac sanchez estrada
carlos morales ortega
cesar yahir perez diaz
christian peña granillo
christopher omar parra grimaldi
cinthya velez grande
daniel aguilar maya
diego andres ramirez garcia
eduardo abdiel mancera hernandez
eduardo alfredo ceron maciel
gabriel alexander carmona garcia
gabriela stephania figueroa villamares
gamaliel rios lira
giovanni salvador chavez villanueva
heber adan pilares garcia
hugo yael juarez garcia
ian pang araizaga
ikechukwu akwueze daniel
ivette alejandra carrillo cervantes
jesica villalpando aguilar
jesus javier jacobo ruiz
jorge antonio lopez chong
jose gerardo flores saavedra
karla andrea velazquez martinez
liliana alejandra muñiz hernandez
luis fabian armenta chora
luis joaquin castro serrato
marco antonio sanchez hernandez
maria fernanda verano peralta
martin gonzalez rico
miguel venegas rocha
rocio garcia lemus
rocio sanchez garcia
rodolfo mario teran garcia
rodrigo gael guzman alburo
samanta ivonne estrada romero
saul garcia lopez
vanessa nava alberto
yaxca alexa quero bautista

## 0.3 Connect to google sheets

In [7]:
gc = gspread.oauth()

In [8]:
#@title Opening excel survey

url_info = "https://docs.google.com/spreadsheets/d/1tvwObZXCbz2UETcD42GujvZ7qXcpkJOJayoXEDYc5mQ/edit?usp=sharing" #@param {"type":"string"}
ss= gc.open_by_url(url_info)
sh = ss.get_worksheet(0)
data  = sh.get_all_values()

## 0.4 Check who did not delivered

In [9]:
# Print columns
print("\n".join([f"{i:02} {col}" for i,col in enumerate(data[0])]))

00 Marca temporal
01 Dirección de correo electrónico
02 Puntuación
03 Nombre completo
04 Nombres trabajo en equipo
05 Anotar cinco cadenas que pertenezcan al lenguaje especificado
06 Anotar cinco cadenas que no pertenezcan al lenguaje especificado
07 Transcribir los pasos del lemma de bombeo
08 Gramática libre de contexto
09 Argumente que la gramática es correcta
10 Nombres trabajo en equipo
11 Anotar cinco cadenas que pertenezcan al lenguaje especificado
12 Anotar cinco cadenas que no pertenezcan al lenguaje especificado
13 Anotar el Autómata de Pila
14 Nombres trabajo en equipo
15 Resumen notación Bacus-Naur 
16 Segmento de if
17 Explique funcionamiento de segmento if
18 Nombres trabajo en equipo
19 Anotar cinco cadenas que pertenezcan al lenguaje especificado
20 Anotar cinco cadenas que no pertenezcan al lenguaje especificado
21 Anotar el Autómata de Pila

In [10]:
errs=0
id2rows={}
for i,row in enumerate(data[1:]):
  res=process.extract(row[3].lower(),lista_.keys(),limit=1)
  if len(res)==0:
    print(f"[red]No match found: {row[3].lower()} [/red]")
    errs+=1
  else:
    res=res[0]
    if res[1]<85:
      print(f"[red]No enough evidence: {row[3].lower()} [/red]")
      errs+=1
    else:
      id2rows[lista_[res[0]]]=row

if errs > 0:
  print("[red]❌ it did not pass[/red]")
else:
  print(f"[green]✓ it seems to be OK, total found {len(id2rows)}[/green]")

✓ it seems to be OK, total found 42

## 0.5 Functions

In [56]:
from maquinas.io import *
from maquinas.regular.ndfa_e import NonDeterministicFiniteAutomaton_epsilon as NDFA_e
from maquinas.regular.ndfa import NonDeterministicFiniteAutomaton as NDFA
from maquinas.regular.dfa import DeterministicFiniteAutomaton as DFA

from maquinas.regular.RE import RegularExpression as RE
from maquinas.contextfree.cfg import ContextFreeGrammar as CFG
from maquinas.parser.earley_parser import print_chart, chart2table


class Responses():

  def __init__(self):
    self.res={}
    self.num={}

  def set_problem(self,num):
    self.num=num
    if not num in self.res:
      self.res[self.num]={}

  def append(self,type,res):
    if not type in self.res[self.num]:
      self.res[self.num][type]=[]
    self.res[self.num][type].append(res)

  def concat(self,type,res):
    if not type in self.res[self.num]:
      self.res[self.num][type]=[]
    self.res[self.num][type]+=res

  def get(self,num,type):
    return self.res[num][type]

def clean_accept_reject(w):
  w=w.strip()
  if w.startswith('w='):
    w=w[2:]
  w=w.strip()
  w=w.replace("-","")
  w=w.replace(",","")
  w=w.replace(";","")
  w=w.replace(".","")
  if w=="Ɛ":
        w=''
  
  return w

re_http=re.compile("(https?://.*)")
re_docs_id=re.compile(r"/d/([^/]+)/")

def find_url(text):
  m=re_http.search(text)
  if m:
    return m.group(1)
  else:
    return None


def load_code(text,type):
  trans=[]
  
  for line in text.splitlines():
    
    line=str(line.strip())
    if len(line)==0:
        continue
    if "States" in line:      
      states=line[10:].split(',')
      states=[a.strip() for a in states]
    if line.startswith("Sigma  "):
      line=line.replace("ε","")
      sigma=line[10:].split(',')
      sigma=[s.strip() for s in sigma]
    if line.startswith("Initial "):
      q0=line[10:].strip()
    if "Aceptors" in line:
      A=line[11:-1].split(',')
      A=[a.strip() for a in A]
    if "Acceptors" in line:
      A=line[12:-1].split(',')
      A=[a.strip() for a in A]
    
    if "→" in line:
      
      bits=line.split("→")
      bits0=bits[0].split(',')
      if type=="NDFA_e":
        if not bits[1].strip()[0]=='{':
          print("[red]It did not found a set in transition of NDFA_e[/red]")
          return None
      if len(bits[1].strip())==2:
        continue
    
      if type=="DFA":
        trans.append(((bits0[0].strip(),bits0[1].strip()),bits[1].strip()))
      else:
          bits1=bits[1].strip()[1:-1].split(',')
          trans.append(((bits0[0].strip(),bits0[1].strip()),[b.strip() for b in bits1]))
      
  if type=="NDFA_e":
    try:
      return NDFA_e(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
    except UnboundLocalError:
      print("[red]One variable not found[/red]")
      return None
  if type=="NDFA":
    try:
      return NDFA(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
    except UnboundLocalError:
      print("[red]One variable not found[/red]")
      return None
  if type=="DFA":
    try:
      return DFA(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
    except UnboundLocalError:
      print("[red]One variable not found[/red]")
      return None
  if type==None:
    
     return NDFA_e(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
     return NDFA(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
     return DFA(Q=states,sigma=sigma,q_0=q0,A=A,delta=trans)
    
    
    
re_transition=re.compile(r"  ([^,\n]+,[^ ]+) →")
re_epsilon=re.compile(r",e →")

def create_fa(text,type):

  # check for url
  url=find_url(text)
  if url:  
    print(f"{url}")
    if 'google' in url:
      m=re_docs_id.search(url)
      if m:
        gid=m.group(1)
    
      if "folder" in url:
        print(f"[red]I can't open a folder:{url}[/red]")
        return None,None
      if 'document' in url:
        if 'edit?' in url:
            bits=url.split("edit?")
            url=bits[0]+"export?format=txt"
        elif 'edit' in url:
            bits=url.split("edit")
            url=bits[0]+"export?format=txt"
            
      gdown.download(url, "tmp.fa", quiet=True, fuzzy=True)
      with open("tmp.fa",'r',encoding='utf8') as f:
        try:
          text=f.read()
        except UnicodeDecodeError:
          print(f"[red]Error with file: {url}[/red]")
          return None, None
      with open("tmp.fa","w") as f:
        pass
    else:
      id=url
      with urllib.request.urlopen(url) as f:
        text = f.read()
  else:
    gid=None


  if not text:
      return None,None
  if "fsa_trans" in text:
    print(f"[red]Editor format: {url}[/red]")
    return gid,None
  if "<?xml" in text:
    #text=text.replace("&#13;","")
    text="<?xml"+text.split("<?xml")[1]
    text=text.split("</structure>")[0]+"</structure>\n"
    try:
        return gid,load_jflap(text)
    except JFLAPXMLError:
        return None,None
  elif "States" in text:
    text=text.replace("Sigma","\nSigma")
    text=text.replace("Initial","\nInitial")
    text=text.replace("Acceptors","\nAcceptors")
    text=text.replace("Aceptors","\nAcceptors")
    text=text.replace("Transitions","\nTransitions")
    text=re_transition.sub(r"\n\1 →",text)
    text=re_epsilon.sub(r",ε →",text)
    return gid,load_code(text,type)
  elif "RE(" in text:
    text=text.replace(".print_summary()","")
    bits=text.split("RE(")
    try:
        return gid,eval("RE("+bits[1])
    except:
        print(text)
  elif "DFA(" in text:
    text=text.replace(".print_summary()","")
    bits=text.split("DFA(")
    try:
        return gid,eval("DFA("+bits[1])
    except:
        print(text)
  elif "|" in text.split('\n')[0]:
      return gid,load_fa(text)

  return None,None

## Section 1
---


In [46]:
solutions=[
    # section 1
    {0: # problema uno
     {"alpabeth":set("0 1".split()),
      "accept":['0001','01','0010','00010','00001'],
      "reject":['1','010','0100','10','001000'],
     },
     1: # problema uno
     {"alpabeth":set("0 1".split()),
      "accept":['0001','10','0100','01000','10000'],
      "reject":['1','010','0010','01','000100'],
     },
     2: # problema uno
     {"alpabeth":set("0 1".split()),
      "accept":['1000','01','0010','00010','00001'],
      "reject":['1','010','0100','10','001000'],
     },
     3: # problema uno
     {"alpabeth":set("0 1".split()),
      "accept":['1000','10','0100','01000','10000'],
      "reject":['1','010','0010','01','000100'],
     },
     4: # problema uno
     {"alpabeth":set("0 1 2 3 4 5 6 7 8 9".split()),
      "accept":['1900','1950','1960','1970','1980','1990','2000','2001','2002','1999'],
      "reject":['10','2022','2021','2010','20'],
     },
    },
    {5: # problema uno
     {"alpabeth":set(['△','◯','⋄','□']),
      "accept":['◯⋄◯','⋄◯⋄◯⋄','△◯△◯△','□◯□◯□','□□□□□□□□'],
      "reject":['△△△','◯','△△△◯','⋄⋄⋄⋄⋄⋄⋄△△△◯','□□□□□□□△△△◯'],
      
     },
     6: # problema uno
     {"alpabeth":set(['a','b','c','d']),
      "accept":['acbbb','aaaaaccc','bbbcbbb','babcb','bcbdb'],
      "reject":['cc','bb','b','bbbb','cccc'],      
     },
     7: # problema uno
     {"alpabeth":set(['x','y','z','w']),
      "accept":['zxyzw','zxyzw','xyzwz','xyzwzwwwwww','xyzwz'],
      "reject":['xyzw','z','zzz','xyzwzz','xyz'],      
     },
     8: # problema uno
     {"alpabeth":set(['i','j','k','l']),
      "accept":['jjji','jjjiii','illl','ijikilll','jjjijikilll'],
      "reject":['i','jjj','lll','jjjii','iillll'],      
     },
    },
    
]    

In [116]:
responses=Responses()
doc2name={}
FB={}
POINTS={}

for num,(id,row) in list(enumerate(id2rows.items())):
  responses.set_problem(assign[id][0])
  problem=assign[id][0]
  # recover name
  name=row[3].strip()
  POINTS[name]=[5,5,3,5,3,10,5,
                5,5,16,
                5,5,5,
                5,5,13]
  FB[name]=["" for _ in POINTS[name]]
  print(f"[yellow]{name}/{num+1} ({problem}) [/yellow]")

  ### Seccion uno
  print("[magenta]SECTION 1[/magenta]")
  accept=[clean_accept_reject(a) for a in row[5].split() if a not in ['•']]
  for a in accept:
      for a_ in a:
          if '}' in a:
            FB[name][0]+=f"Cadena '{a}' entre corchetes no hace sentido (0.5 punto menos)\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
          elif not a_ in solutions[0][problem]['alpabeth']:
            FB[name][0]+=f"Cadena '{a}' contine símbolo '{a_}' que no está en al alfabeto designado (punto menos)\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
            break
      a_=a
      
      if problem==0:
        bits=a_.split('1')
        if len(bits)!=2:
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
            break

        if not (len(bits[0])>len(bits[1]) or (len(bits[0])==3 and len(bits[1])==0)):
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
      if problem==1:
        bits=a_.split('0')
        if len(bits)!=2:
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
            break

        if not (len(bits[0])<len(bits[1]) or (len(bits[0])==3 and len(bits[1])==0)):
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
      if problem==2:
        bits=a_.split('1')
        if len(bits)!=2:
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
            break

        if not ((len(bits[0])>len(bits[1])) or (len(bits[0])==0 and len(bits[1])==3)):
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
      if problem==3:
        bits=a_.split('0')
        if len(bits)!=2:
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
            break

        if not (len(bits[0])<len(bits[1]) or (len(bits[0])==0 and len(bits[1])==3)):
            FB[name][0]+=f"Cadena '{a}' no debe ser aceptada\n"
            POINTS[name][0]=max(0,POINTS[name][0]-1)
        
            
  reject=[clean_accept_reject(r) for r in row[6].split() if r not in ['•{}']]

  for a in reject:
      for a_ in a:
          if '}' in a:
            FB[name][1]+=f"Cadena '{a}' entre corchetes no hace sentido (0.5 punto menos)\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
          elif not a_ in solutions[0][problem]['alpabeth']:
            FB[name][1]+=f"Cadena '{a}' contine símbolo '{a_}' que no está en al alfabeto designado (punto menos)\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
            break
      a_=a
      
      if problem==0:
        bits=a_.split('1')
        if len(bits)!=2:
            break

        if (len(bits[0])>len(bits[1]) or (len(bits[0])==3 and len(bits[1])==0)):
            FB[name][1]+=f"Cadena '{a}' debe ser aceptada\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
      if problem==1:
        bits=a_.split('0')
        if len(bits)!=2:
             break

        if (len(bits[0])<len(bits[1]) or (len(bits[0])==3 and len(bits[1])==0)):
            FB[name][1]+=f"Cadena '{a}' debe ser aceptada\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
      if problem==2:
        bits=a_.split('1')
        if len(bits)!=2:
            break

        if ((len(bits[0])>len(bits[1])) or (len(bits[0])==0 and len(bits[1])==3)):
            FB[name][1]+=f"Cadena '{a}' debe ser aceptada\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
      if problem==3:
        bits=a_.split('0')
        if len(bits)!=2:
            break

        if (len(bits[0])<len(bits[1]) or (len(bits[0])==0 and len(bits[1])==3)):
            FB[name][1]+=f"Cadena '{a}' debe ser aceptada\n"
            POINTS[name][1]=max(0,POINTS[name][1]-1)
            
            
      if len(row[7].strip())==0:
            FB[name][2]+=f"No se definió parte de la cadena dependiente de n\n"
            POINTS[name][2]=max(0,POINTS[name][2]-1)
        
    
            FB[name][3]+=f"No se particionó la cadena\n"
            POINTS[name][3]=max(0,POINTS[name][3]-1)
            
            FB[name][4]+=f"No se aplicó el lema del bombeo\n"
            POINTS[name][4]=max(0,POINTS[name][4]-1)
            
      if len(row[9].strip())==0:
            FB[name][6]+=f"No se argumentó la gramática\n"
            POINTS[name][6]=max(0,POINTS[name][6]-1)
  ps=[]
  g_=re.sub(r'\{.*\}',"",row[8])
  print(g_)
  for r in re.split('\n+', g_):
        r=r.replace('donde P:','')
        r=r.replace('Donde P:','')
        r=r.replace('P:','')
        r=r.replace('(, P,S)','')
        r=r.replace('G=','')
        
        if not len(r)>0:
            continue
        print(r)
        h,bs=r.strip().split('->',1)
        bs=bs.split('|')
        for b in bs:
            ps.append(f"{h.strip()} -> {b.strip()}")
  g="; ".join(ps)

  grammar_1=CFG(g)
    

  for a in solutions[0][problem]['accept']:
    if not grammar_1.accepts(a):
        FB[name][5]+=f"No se genera la cadena '{a}' en la gramática\n"
        POINTS[name][5]=max(0,POINTS[name][5]-1)

  for a in solutions[0][problem]['reject']:
    if grammar_1.accepts(a):
        FB[name][5]+=f"Se genera la cadena '{a}' en la gramática que no se debería \n"
        POINTS[name][5]=max(0,POINTS[name][5]-1)
    
    
            
  ### PRINTS RESULTS
  total=0
  for i,(fb,p) in enumerate(zip(FB[name],POINTS[name])):
        print(f"{i}: {p}",fb)
        total+=p
  print("Total resultados:", total)

Villalpando Aguilar Jesica/1 (2) 

SECTION 1

S -> 1000 | 0G
G -> 0G | 0G0 | 1

S -> 1000 | 0G

G -> 0G | 0G0 | 1

...

Hugo Yael Juárez García/38 (0) 

SECTION 1

G:[,P,S]      P:{ S -> 0B; B -> 0B0 B -> 0B B-> 1

G:[,P,S]      { S -> 0B; B -> 0B0 B -> 0B B-> 1

FailedLeftRecursion: (1:1) infinite left recursion :
G:[,P,S]      { S -> 0B; B -> 0B0 B -> 0B B-> 1
^
productions
start